In [12]:
import pandas as pd

# Load the dataset
df = pd.read_parquet("C:/Users/omp72/Downloads/yellow_tripdata_2023-01.parquet")

# View basic info
print("Original shape:", df.shape)
print("Columns:", df.columns)

# Step 1: Drop rows with any missing/null values
df.dropna(inplace=True)

# Step 2: Filter out trips with invalid or unrealistic values
df = df[
    (df['passenger_count'] > 0) &
    (df['trip_distance'] > 0) &
    (df['fare_amount'] > 0) &
    (df['tpep_pickup_datetime'] < df['tpep_dropoff_datetime'])  # drop if pickup is after dropoff
]

# Step 3: Convert datetime columns
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Step 4: Create new useful columns
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['trip_duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Step 5: Remove trips longer than 3 hours (outliers)
df = df[df['trip_duration_minutes'] <= 180]

# Optional: Reset index after filtering
df.reset_index(drop=True, inplace=True)

# Final overview
print("Cleaned shape:", df.shape)
print(df.head())


Original shape: (3066766, 19)
Columns: Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
Cleaned shape: (2881179, 22)
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
4         2  2023-01-01 00:50:34   2023-01-01 01:02:52              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97  